In [2]:
import pandas as pd

# Preparação dos dados de Unidades de Saúde do Datasus

In [32]:
# Carregando tabela do datasus
df = pd.read_csv('tbEstabelecimento202106.csv', sep=';', low_memory=False)


# Filtrando seguintes estabelecimentos:
# 1: Unidade Básica de Saúde; 
# 25: Centro de Imunização
df = df[df['CO_TIPO_ESTABELECIMENTO'].isin([1, 25])]


# Filtrando e renomeando seguintes colunas:
columns = [
  'NO_FANTASIA', 
  'CO_MUNICIPIO_GESTOR',
  'NO_LOGRADOURO', 
  'NU_ENDERECO',
  'NO_COMPLEMENTO',
  'NO_BAIRRO',
  'CO_CEP',
  'NU_TELEFONE',
  'NU_LATITUDE',
  'NU_LONGITUDE'
]

alias = [
  'nome',
  'cidade',
  'logradouro',
  'numero',
  'complemento',
  'bairro',
  'cep',
  'telefone',
  'latitude',
  'longitude'
]

df = df[columns]
mapper = {c: a for c, a in zip(columns, alias)}
df = df.rename(columns=mapper)


# Sanitização de dados: removendo registros com 
# valores inválidos de latitude ou longitude
df = df[~(df.latitude.isna() | df.longitude.isna())]


In [38]:
# Adição do nome da cidade e unidade federativa
df_cidade = pd.read_csv('tbMunicipio202106.csv', sep=';')

get_nome_cidade = lambda x: df_cidade.loc[df_cidade.CO_MUNICIPIO == x, 'NO_MUNICIPIO'].item()
get_uf = lambda x: df_cidade.loc[df_cidade.CO_MUNICIPIO == x, 'CO_SIGLA_ESTADO'].item()

df['uf'] = [get_uf(cod_cidade) for cod_cidade in df['cidade']]
df['cidade'] = [get_nome_cidade(cod_cidade) for cod_cidade in df['cidade']]

In [44]:
# Salvando dados em um novo arquivo .csv
df.to_csv('datasus_unidades.csv', index=False)